In [2]:
import pandas as pd
from shapely.ops import unary_union
import shapely
import geopandas as gpd
from shapely.geometry import Polygon
from shapely.geometry import Point
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import requests
import re
import glob
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt


def shp_hill (name_of_source_file, name_of_cat_file, name_of_result_file, epsilon):
    """
    @ author:                  Shervan Gharari
    @ Github:                  https://github.com/ShervanGharari/shapefile_standardization
    @ author's email id:       sh.gharari@gmail.com
    @license:                  MIT

    This function gets name of a shapefile and remove inernal holes

    Arguments
    ---------
    name_of_source_file: string, the name of the source file including path and extension
    name_of_cat_file: string, the name of the corresponding catchment (subbasin)
        for the unresolved hills
    name_of_result_file: string, the name of the file that includes fixed shapes
        including path and extension
    epsilon
    
    Returns
    -------

    Saves Files
    -------
    a shp file that includes corrected polygones
    a possible shapefile that includes the fixed shapes
    """
    
    shp = gpd.read_file(name_of_source_file)
    cat = gpd.read_file(name_of_cat_file)
    shp_all = None

    ## STEP1, load a shapefile, and find its intesection with itself. there are
    ## there should be some holes in the shapefile. the holes are given as a separaete shape
    shp_temp = gpd.overlay(shp, shp, how='intersection')
    shp_temp = shp_temp [shp_temp.FID_1 != shp_temp.FID_2]
    shp_temp = shp_temp.reset_index()    
    shp_temp = gpd.overlay(shp, shp_temp, how='difference')
    shp_temp = shp_temp.reset_index()
    
    ## STEP2, remove possible cat from the unresolved costal hillslope
    shp_temp = gpd.overlay(shp_temp, cat, how='difference')
    shp_temp = shp_temp.reset_index()
    
    ## STEP3, break the touching polygons into separate polygons, remove the links (lines)
    # shp_temp = shp_temp.buffer(-epsilon).buffer(epsilon)
    shp_temp = shp_temp.buffer(-epsilon, cap_style=2, join_style=2)
    shp_temp = gpd.GeoDataFrame(shp_temp)
    shp_temp.columns = ['geometry'] # rename the colomn to geometry
    shp_temp.to_file('temp4.shp')

    ## STEP4, break the polygones into separete shape in a shapefile
    shp = gpd.read_file('temp4.shp')
    for index, _ in shp.iterrows():
        polys = shp.geometry.iloc[index] # get the shape
        if polys is not None:
            if polys.type == 'Polygon':
                shp_temp = gpd.GeoSeries(polys) # convert multipolygon to a shapefile with polygons only
                shp_temp = gpd.GeoDataFrame(shp_temp) # convert multipolygon to a shapefile with polygons
                shp_temp.columns = ['geometry'] # naming geometry column
            if polys.type == 'MultiPolygon':
                shp_temp = gpd.GeoDataFrame(polys) # convert multipolygon to a shapefile with polygons 
                shp_temp.columns = ['geometry'] # naming geometry column
            if shp_all is None:
                shp_all = shp_temp
            else:
                shp_all = shp_all.append(shp_temp)
    shp_all = shp_all.buffer(epsilon, cap_style=2, join_style=2)
    shp_all.to_file(name_of_result_file)
    shp_all = gpd.read_file(name_of_result_file)
    # add COMIDS
    shp_all['COMID'] = np.arange(shp_all.shape[0])+max(cat.COMID)+1
    shp_all = shp_all.set_crs("EPSG:4326")
    shp_all = shp_all.drop(columns=['FID'])
    shp_all.to_file(name_of_result_file)


def intersection_shp(shp_1, shp_2):
    """
    @ author:                  Shervan Gharari
    @ Github:                  https://github.com/ShervanGharari/candex
    @ author's email id:       sh.gharari@gmail.com
    @license:                  Apache2
    This fucntion intersect two shapefile. It keeps the fiels from the first and second shapefiles (identified by S_1_ and 
    S_2_). It also creats other field including AS1 (area of the shape element from shapefile 1), IDS1 (an arbitary index
    for the shapefile 1), AS2 (area of the shape element from shapefile 1), IDS2 (an arbitary index for the shapefile 1), 
    AINT (the area of teh intersected shapes), AP1 (the area of the intersected shape to the shapes from shapefile 1),
    AP2 (the area of teh intersected shape to the shapefes from shapefile 2), AP1N (the area normalized in the case AP1
    summation is not 1 for a given shape from shapefile 1, this will help to preseve mass if part of the shapefile are not 
    intersected), AP2N (the area normalized in the case AP2 summation is not 1 for a given shape from shapefile 2, this
    will help to preseve mass if part of the shapefile are not intersected)
    
    Arguments
    ---------
    shp1: geo data frame, shapefile 1
    shp2: geo data frame, shapefile 2
    
    Returns
    -------
    result: a geo data frame that includes the intersected shapefile and area, percent and normalized percent of each shape
    elements in another one
    """
    # Calculating the area of every shapefile (both should be in degree or meters)
    column_names = shp_1.columns
    column_names = list(column_names)

    # removing the geometry from the column names
    column_names.remove('geometry')

    # renaming the column with S_1
    for i in range(len(column_names)):
        shp_1 = shp_1.rename(
            columns={column_names[i]: 'S_1_' + column_names[i]})

    column_names = shp_2.columns
    column_names = list(column_names)

    # removing the geometry from the colomn names
    column_names.remove('geometry')

    # renaming the column with S_2
    for i in range(len(column_names)):
        shp_2 = shp_2.rename(
            columns={column_names[i]: 'S_2_' + column_names[i]})

    # Caclulating the area for shp1
    shp_1['AS1'] = shp_1.area
    shp_1['IDS1'] = np.arange(shp_1.shape[0])+1

    # Caclulating the area for shp2
    shp_2['AS2'] = shp_2.area
    shp_2['IDS2'] = np.arange(shp_2.shape[0])+1

    # making intesection
    result = spatial_overlays (shp_1, shp_2, how='intersection')

    # Caclulating the area for shp2
    result['AINT'] = result['geometry'].area
    result['AP1'] = result['AINT']/result['AS1']
    result['AP2'] = result['AINT']/result['AS2']
    
    
    # taking the part of data frame as the numpy to incread the spead
    # finding the IDs from shapefile one
    ID_S1 = np.array (result['IDS1'])
    AP1 = np.array(result['AP1'])
    AP1N = AP1 # creating the nnormalized percent area
    ID_S1_unique = np.unique(ID_S1) #unique idea
    for i in ID_S1_unique:
        INDX = np.where(ID_S1==i) # getting the indeces
        AP1N[INDX] = AP1[INDX] / AP1[INDX].sum() # normalizing for that sum
        
    # taking the part of data frame as the numpy to incread the spead
    # finding the IDs from shapefile one
    ID_S2 = np.array (result['IDS2'])
    AP2 = np.array(result['AP2'])
    AP2N = AP2 # creating the nnormalized percent area
    ID_S2_unique = np.unique(ID_S2) #unique idea
    for i in ID_S2_unique:
        INDX = np.where(ID_S2==i) # getting the indeces
        AP2N[INDX] = AP2[INDX] / AP2[INDX].sum() # normalizing for that sum
        
    result ['AP1N'] = AP1N
    result ['AP2N'] = AP2N
        
    return result


def spatial_overlays(df1, df2, how='intersection', reproject=True):
    """Perform spatial overlay between two polygons.
    Currently only supports data GeoDataFrames with polygons.
    Implements several methods that are all effectively subsets of
    the union.
    
    Omer Ozak
    ozak
    https://github.com/ozak
    https://github.com/geopandas/geopandas/pull/338
    Parameters
    ----------
    df1 : GeoDataFrame with MultiPolygon or Polygon geometry column
    df2 : GeoDataFrame with MultiPolygon or Polygon geometry column
    how : string
        Method of spatial overlay: 'intersection', 'union',
        'identity', 'symmetric_difference' or 'difference'.
    use_sindex : boolean, default True
        Use the spatial index to speed up operation if available.
    Returns
    -------
    df : GeoDataFrame
        GeoDataFrame with new set of polygons and attributes
        resulting from the overlay
    """
    df1 = df1.copy()
    df2 = df2.copy()
    df1['geometry'] = df1.geometry.buffer(0)
    df2['geometry'] = df2.geometry.buffer(0)
    if df1.crs!=df2.crs and reproject:
        print('Data has different projections.')
        print('Converted data to projection of first GeoPandas DatFrame')
        df2.to_crs(crs=df1.crs, inplace=True)
    if how=='intersection':
        # Spatial Index to create intersections
        spatial_index = df2.sindex
        df1['bbox'] = df1.geometry.apply(lambda x: x.bounds)
        df1['sidx']=df1.bbox.apply(lambda x:list(spatial_index.intersection(x)))
        pairs = df1['sidx'].to_dict()
        nei = []
        for i,j in pairs.items():
            for k in j:
                nei.append([i,k])
        pairs = gpd.GeoDataFrame(nei, columns=['idx1','idx2'], crs=df1.crs)
        pairs = pairs.merge(df1, left_on='idx1', right_index=True)
        pairs = pairs.merge(df2, left_on='idx2', right_index=True, suffixes=['_1','_2'])
        pairs['Intersection'] = pairs.apply(lambda x: (x['geometry_1'].intersection(x['geometry_2'])).buffer(0), axis=1)
        pairs = gpd.GeoDataFrame(pairs, columns=pairs.columns, crs=df1.crs)
        cols = pairs.columns.tolist()
        cols.remove('geometry_1')
        cols.remove('geometry_2')
        cols.remove('sidx')
        cols.remove('bbox')
        cols.remove('Intersection')
        dfinter = pairs[cols+['Intersection']].copy()
        dfinter.rename(columns={'Intersection':'geometry'}, inplace=True)
        dfinter = gpd.GeoDataFrame(dfinter, columns=dfinter.columns, crs=pairs.crs)
        dfinter = dfinter.loc[dfinter.geometry.is_empty==False]
        dfinter.drop(['idx1','idx2'], inplace=True, axis=1)
        return dfinter
    elif how=='difference':
        spatial_index = df2.sindex
        df1['bbox'] = df1.geometry.apply(lambda x: x.bounds)
        df1['sidx']=df1.bbox.apply(lambda x:list(spatial_index.intersection(x)))
        df1['new_g'] = df1.apply(lambda x: reduce(lambda x, y: x.difference(y).buffer(0), 
                                 [x.geometry]+list(df2.iloc[x.sidx].geometry)) , axis=1)
        df1.geometry = df1.new_g
        df1 = df1.loc[df1.geometry.is_empty==False].copy()
        df1.drop(['bbox', 'sidx', 'new_g'], axis=1, inplace=True)
        return df1
    elif how=='symmetric_difference':
        df1['idx1'] = df1.index.tolist()
        df2['idx2'] = df2.index.tolist()
        df1['idx2'] = np.nan
        df2['idx1'] = np.nan
        dfsym = df1.merge(df2, on=['idx1','idx2'], how='outer', suffixes=['_1','_2'])
        dfsym['geometry'] = dfsym.geometry_1
        dfsym.loc[dfsym.geometry_2.isnull()==False, 'geometry'] = dfsym.loc[dfsym.geometry_2.isnull()==False, 'geometry_2']
        dfsym.drop(['geometry_1', 'geometry_2'], axis=1, inplace=True)
        dfsym = gpd.GeoDataFrame(dfsym, columns=dfsym.columns, crs=df1.crs)
        spatial_index = dfsym.sindex
        dfsym['bbox'] = dfsym.geometry.apply(lambda x: x.bounds)
        dfsym['sidx'] = dfsym.bbox.apply(lambda x:list(spatial_index.intersection(x)))
        dfsym['idx'] = dfsym.index.values
        dfsym.apply(lambda x: x.sidx.remove(x.idx), axis=1)
        dfsym['new_g'] = dfsym.apply(lambda x: reduce(lambda x, y: x.difference(y).buffer(0), 
                         [x.geometry]+list(dfsym.iloc[x.sidx].geometry)) , axis=1)
        dfsym.geometry = dfsym.new_g
        dfsym = dfsym.loc[dfsym.geometry.is_empty==False].copy()
        dfsym.drop(['bbox', 'sidx', 'idx', 'idx1','idx2', 'new_g'], axis=1, inplace=True)
        return dfsym
    elif how=='union':
        dfinter = spatial_overlays(df1, df2, how='intersection')
        dfsym = spatial_overlays(df1, df2, how='symmetric_difference')
        dfunion = dfinter.append(dfsym)
        dfunion.reset_index(inplace=True, drop=True)
        return dfunion
    elif how=='identity':
        dfunion = spatial_overlays(df1, df2, how='union')
        cols1 = df1.columns.tolist()
        cols2 = df2.columns.tolist()
        cols1.remove('geometry')
        cols2.remove('geometry')
        cols2 = set(cols2).intersection(set(cols1))
        cols1 = list(set(cols1).difference(set(cols2)))
        cols2 = [col+'_1' for col in cols2]
        dfunion = dfunion[(dfunion[cols1+cols2].isnull()==False).values]
        return dfunion

def intersect(name_of_source_file,
              name_of_ERA_file,
              name_of_result_file,
              field_ID,
              dic_rename):
    """
    @ author:                  Shervan Gharari
    @ Github:                  https://github.com/ShervanGharari/shapefile_standardization
    @ author's email id:       sh.gharari@gmail.com
    @license:                  MIT

    This function gets name of a shapefile, its directory, and its extensions (such as gpkg or shp) and
    save a stadard shapefile. if presence it also save the holes of a shapefile

    Arguments
    ---------
    name_of_source_file: string, the name of the source file including path and extension
    name_of_ERA_file: string, the name of the final file including path and extension
    name_of_result_file: string, the name of the file that includes holes including path
        and extension
    field_ID: string, the name of the field in the original shapefile that is used for keeping
        track of holes
    dic_rename: float; the tolerance to compare area before and after correction and report
        differences

    Returns
    -------


    Saves Files
    -------
    a shp file that includes corrected polygones
    a possible shapefile that includes the removed problematice holes
    a log file in the same folder descringin the invalid shapefiles
    """
    
    shp1 = gpd.read_file (name_of_source_file)
    shp1.crs = 'epsg:4326'
    shp1["lon_c"] = shp1.centroid.x # pass calculated centroid lon to the shp1
    shp1["lat_c"] = shp1.centroid.y # pass calculated centroid lat to the shp1
    shp2 = gpd.read_file (name_of_ERA_file)
    shp_int = intersection_shp(shp1, shp2)
    shp_int = shp_int.rename(columns=dic_rename) # weight of each ERA5 grid in subbasin
    shp_int = shp_int.sort_values(by=[field_ID])
    shp_int.to_file(name_of_result_file)
    


# Hillslope correction
### # there is no 49 hillslope and there is only one shape in hillslope 36

In [ ]:
# the 2 digit pfaf code for the shapefile to be processed
# list of IDs for downloading the processing
IDs = ['11', '12', '13', '14', '15', '16', '17', '18',
       '21', '22', '23', '24', '25', '26', '27', '28', '29',
       '31', '32', '33', '34', '35',
       '41', '42', '43', '44', '45', '46', '47', '48',
       '51', '52', '53', '54', '55', '56', '57',
       '61', '62', '63', '64', '65', '66', '67',
       '71', '72', '73', '74', '75', '76', '77', '78',
       '81', '82', '83', '84', '85', '86',
       '91']
# in this folder create subfolders cat, riv, hill, cat_step_1,cat_step_2
path = '/Users/shg096/Desktop/MERIT_Hydro/'
# break the hillslopes into separaete hillslopes between the river segments
for ID in IDs:
    shp_hill (path+'hill/hillslope_'+ID+'_clean.shp',
              path+'cat/cat_pfaf_'+ID+'_MERIT_Hydro_v07_Basins_v01_bugfix1.shp',
              path+'hill_step_0/hillslope_'+ID+'_clean_corr1_test.shp',
              0.0000001)

<ipython-input-2-9b5468c6d22b>:51: UserWarning: `keep_geom_type=True` in overlay resulted in 1936 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  shp_temp = gpd.overlay(shp, shp, how='intersection')
<ipython-input-2-9b5468c6d22b>:58: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None

  shp_temp = gpd.overlay(shp_temp, cat, how='difference')
<ipython-input-2-9b5468c6d22b>:63: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  shp_temp = shp_temp.buffer(-epsilon, cap_style=2, join_style=2)


# Adding the COMID to the decomposed unresolved hillslopes
## the COMIDs are inclluding both mainland and islands

In [ ]:
IDs = ['11', '12', '13', '14', '15', '16', '17', '18',
       '21', '22', '23', '24', '25', '26', '27', '28', '29',
       '31', '32', '33', '34', '35', '36',
       '41', '42', '43', '44', '45', '46', '47', '48',
       '51', '52', '53', '54', '55', '56', '57',
       '61', '62', '63', '64', '65', '66', '67',
       '71', '72', '73', '74', '75', '76', '77', '78',
       '81', '82', '83', '84', '85', '86',
       '91'] # copy past hillslope 36 to the folder with name _corr1

# add COMID ID to modified hillslope
for ID in IDs:
    shp = gpd.read_file(path+'hill_step_0/hillslope_'+ID+'_clean_corr1_test.shp')
    cat = gpd.read_file(path+'cat/cat_pfaf_'+ID+'_MERIT_Hydro_v07_Basins_v01_bugfix1.shp')
    shp ['COMID'] = np.arange(shp.shape[0])+max(cat.COMID)+1
    shp.to_file(path+'hill_fixed/hillslope_'+ID+'_clean_fixed_test.shp')
    

# assign WGS84 projection to the fixed shapefiles

In [ ]:
# define projection for the corrected hillslopes
for ID in IDs:
    shp = gpd.read_file(path+'hill_fixed/hillslope_'+ID+'_clean_fixed_test.shp')
    shp.crs = {'init': 'epsg:4326', 'no_defs': True}
    shp.to_file(path+'hill_fixed/hillslope_'+ID+'_clean_fixed_test.shp')

# Clean up the colomn names

In [ ]:
# define projection for the corrected hillslopes
for ID in IDs:
    shp = gpd.read_file(path+'hill_fixed/hillslope_'+ID+'_clean_fixed_test.shp')
    shp = shp.drop(columns=['FID'])
    shp.to_file(path+'hill_fixed/hillslope_'+ID+'_clean_fixed_test.shp')

# calculating the area only for north america

In [ ]:
IDs = ['71', '72', '73', '74', '75', '76', '77', '78',
       '81', '82', '83', '84', '85', '86'] # of north america

# define projection for the corrected hillslopes
for ID in IDs:
    shp = gpd.read_file(path+'hill_fixed/hillslope_'+ID+'_clean_fixed.shp')
    shp_temp = shp.to_crs("EPSG:6933") # projection for north america
    shp_temp['unitarea'] = shp_temp.area
    shp['unitarea'] = shp_temp['unitarea']/1000000 # meter2 to km2
    shp.to_file(path+'hill_fixed/hillslope_'+ID+'_clean_fixed_area.shp')
    